In [16]:
import os
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import joblib

# Create models directory if it does not exist
models_dir = r'D:\kowsi\project_works\Advanced_Forecasting_Anomaly_Detection\models'
os.makedirs(models_dir, exist_ok=True)

# Load engineered features
df = pd.read_csv(r'D:\kowsi\project_works\Advanced_Forecasting_Anomaly_Detection\outputs\engineered_features.csv')
df['date'] = pd.to_datetime(df['date'])

# Define feature list without 'region' since it will be one-hot encoded
features = [
    'units_produced_lag1',
    'power_kwh_lag1',
    'units_produced_roll7',
    'power_kwh_roll7',
    'day_of_week',
    'month',
    'week_of_year',
    'commissioned_year',
    'shift_hours_per_day'
]

target_units = 'units_produced'
target_power = 'power_kwh'

# One-hot encode 'region' categorical variable
df = pd.get_dummies(df, columns=['region'])

# Get list of region dummy columns
region_cols = [col for col in df.columns if col.startswith('region_')]

# Prepare feature matrices
X = df[features + region_cols]
y_units = df[target_units]
y_power = df[target_power]

# Time series cross-validation setup
tscv = TimeSeriesSplit(n_splits=5)

def evaluate_model(X, y):
    mae_list = []
    mape_list = []
    for train_idx, val_idx in tscv.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)

        preds = model.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        mape = mean_absolute_percentage_error(y_val, preds)
        mae_list.append(mae)
        mape_list.append(mape)

    mean_mae = sum(mae_list) / len(mae_list)
    mean_mape = sum(mape_list) / len(mape_list)
    return mean_mae, mean_mape

# Evaluate models
units_mae, units_mape = evaluate_model(X, y_units)
print(f"Units Produced Forecast - MAE: {units_mae:.2f}, MAPE: {units_mape:.2%}")

power_mae, power_mape = evaluate_model(X, y_power)
print(f"Power kWh Forecast - MAE: {power_mae:.2f}, MAPE: {power_mape:.2%}")

# Train final models on full data
units_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)
units_model.fit(X, y_units)

power_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)
power_model.fit(X, y_power)

# Save models
joblib.dump(units_model, os.path.join(models_dir, 'xgb_units_model.pkl'))
joblib.dump(power_model, os.path.join(models_dir, 'xgb_power_model.pkl'))

print("Models trained and saved successfully.")


Units Produced Forecast - MAE: 109.73, MAPE: 8.59%
Power kWh Forecast - MAE: 542.02, MAPE: 11.71%
Models trained and saved successfully.
